<a href="https://colab.research.google.com/github/khanfawaz/LLM/blob/main/TextGeneration_using_GeminiPro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generate text responses from text inputs

### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

In [ ]:
!pip install -q -U google-generativeai

### Import libraries
Import the necessary libraries.

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In [ ]:
# use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## List models

Now you're ready to call the Gemini API. Use `list_models` to see the available Gemini models:

* `gemini-pro`: optimized for text-only prompts.
* `gemini-pro-vision`: optimized for text-and-images prompts.

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-pro
models/gemini-pro-vision


## Generate text from text inputs

For text-only prompts, use the `gemini-pro` model:

In [ ]:
model = genai.GenerativeModel('gemini-pro')

The `generate_content` method can handle a wide variety of use cases, including multi-turn chat and multimodal input, depending on what the underlying model supports. The available models only support text and images as input, and text as output.

In the simplest case, you can pass a prompt string to the `GenerativeModel.generate_content` method:

In [ ]:
%%time
response = model.generate_content("How to learn Large Language Model quickly and efficiently?")

CPU times: user 131 ms, sys: 14.2 ms, total: 146 ms
Wall time: 11.8 s


In simple cases, the `response.text` accessor is all you need. To display formatted Markdown text, use the `to_markdown` function:

In [ ]:
to_markdown(response.text)

> 1. **Understand the Basics First:**
>    - Start by building a solid foundation by understanding the fundamental concepts and principles behind Large Language Models (LLMs). Read introductory articles, tutorials, and research papers to grasp the basics.
> 
> 
> 2. **Choose a Framework or Library:**
>    - Select a popular framework or library for LLM development, such as PyTorch, TensorFlow, or Hugging Face Transformers. Familiarize yourself with its syntax and APIs.
> 
> 
> 3. **Explore Pre-Trained Models:**
>    - Begin by working with pre-trained LLMs provided by platforms like Hugging Face or Google AI. Experiment with their capabilities and usage.
> 
> 
> 4. **Diving into Fine-Tuning:**
>    - Learn how to fine-tune pre-trained LLMs to tailor them to your specific tasks or domains. This is a crucial skill for customizing LLMs for your applications.
> 
> 
> 5. **Understand LLM Architecture:**
>    - Research how Transformer-based architectures, like BERT, GPT, and RoBERTa, are constructed and function.
> 
> 
> 6. **Train Your Own LLM:**
>    - Once you feel comfortable, try training your own LLM from scratch using a suitable dataset. It's essential to start with a smaller model and gradually scale up as your expertise grows.
> 
> 
> 7. **Study the Mathematics Behind LLMs:**
>    - Delve into the mathematics underpinning LLMs to understand their inner workings. This will help you troubleshoot issues more effectively.
> 
> 
> 8. **Practice and Experiment:**
>    - Like any skill, practice is paramount. Experiment with different tasks, datasets, and model configurations to develop a deeper understanding of LLMs.
> 
> 
> 9. **Join Communities and Forums:**
>    - Engage with the active LLM community by joining relevant forums, Slack groups, and online communities. Ask questions and participate in discussions to broaden your knowledge.
> 
> 
> 10. **Contribute to Open-Source Projects:**
>    - Consider contributing to open-source LLM projects or repositories. Working on real-world projects can provide valuable insights and practical experience.
> 
> 
> 11. **Stay Updated with Research:**
>    - Keep abreast of the latest advancements in LLM research by reading academic papers and following influential researchers on social media.
> 
> 
> 12. **Test Your Skills with Competitions:**
>    - Participate in LLM-focused competitions or challenges to showcase your skills and learn from others. These competitions provide opportunities for applying your knowledge in real-world scenarios.
> 
> 
> 13. **Explore LLM Applications:**
>    - Research and explore various applications of LLMs, such as text summarization, language translation, sentiment analysis, and conversational AI.
> 
> 
> 14. **Attend Workshops or Courses:**
>    - Consider attending specialized workshops or online courses dedicated to LLMs. These can provide structured learning experiences and in-depth insights.
> 
> 
> 15. **Build Your Portfolio:**
>    - Create a portfolio showcasing your LLM-related projects, code snippets, and written tutorials. A well-curated portfolio can demonstrate your skills to potential employers or collaborators.

If the API failed to return a result, use `GenerateContentRespose.prompt_feedback` to see if it was blocked due to saftey concerns regarding the prompt.

In [ ]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

Gemini can generate multiple possible responses for a single prompt. These possible responses are called `candidates`, and you can review them to select the most suitable one as the response.

View the response candidates with `GenerateContentResponse.candidates`:

In [ ]:
response.candidates

[content {
  parts {
    text: "Jupiter"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

By default, the model returns a response after completing the entire generation process. You can also stream the response as it is being generated, and the model will return chunks of the response as soon as they are generated.

To stream responses, use `GenerativeModel.generate_content(..., stream=True)`.

In [ ]:
%%time
response = model.generate_content("How to learn Large Language Model quickly and efficiently?", stream=True)

CPU times: user 119 ms, sys: 21.1 ms, total: 140 ms
Wall time: 11.1 s


In [ ]:
for chunk in response:
  print(chunk.text)
  print("_"*80)

1. **Learn the Basics of Natural Language Processing (NLP)**

  -
________________________________________________________________________________
 Understand concepts like tokenization, stemming, lemmatization, stop words, and part-of-speech tagging.
  - Familiarize yourself with different NLP
________________________________________________________________________________
 tasks, such as text classification, named entity recognition, question answering, and machine translation.

2. **Study Deep Learning Algorithms and Architectures**

  - Learn about neural networks, including convolutional neural networks (CNNs), recurrent neural networks (RNNs), and transformers.
  - Understand the concepts of attention
________________________________________________________________________________
 mechanisms, self-attention, and positional encoding.
  - Familiarize yourself with pre-trained language models, such as BERT, GPT, and XLNet.

3. **Set Up Your Development Environment**

  - Choose a p

When streaming, some response attributes are not available until you've iterated through all the response chunks. This is demonstrated below:

In [ ]:
response = model.generate_content("How to learn Large Language Model quickly and efficiently?", stream=True)

The `prompt_feedback` attribute works:

In [ ]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

But attributes like `text` do not:

In [ ]:
try:
  response.text
except Exception as e:
  print(f'{type(e).__name__}: {e}')

IncompleteIterationError: Please let the response complete iteration before accessing the final accumulated
attributes (or call `response.resolve()`)
